In [7]:
import pyscf
import pyscf.tools

In [2]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 Fe -1.67785607 -0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

In [3]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5351.41478186842
HOMO (B) = 0.318236980196613  LUMO (A) = 0.320979897364177
cycle= 1 E= -5347.0130535218  delta_E=  4.4  |g|= 4.93  |ddm|= 11.8
HOMO (B) = -0.208577216881654  LUMO (A) = 0.00879930358562003
cycle= 2 E= -5326.86878641886  delta_E= 20.1  |g|= 8.88  |ddm|= 13.1
HOMO (B) = 1.25634475236875  LUMO (B) = 0.0701291625721957
cycle= 3 E= -5328.84109730652  delta_E= -1.97  |g|= 5.69  |ddm|= 29.9
HOMO (A) = 0.189978800807605  LUMO (B) = 0.322966982029677
cycle= 4 E= -5355.41569900171  delta_E= -26.6  |g|= 0.

In [4]:
F = mf.get_fock()

In [5]:
import numpy as np
import scipy
import copy as cp
import math

def get_frag_bath(Pin, frag, S, thresh=1e-7, verbose=1):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Xinv = scipy.linalg.inv(X)

    Nbas = S.shape[0]
    Cfrag = Xinv@np.eye(Nbas)[:,frag]
    

    P = X@Pin@X.T

    nfrag = np.trace(P[frag,:][:,frag])
    P[frag,:] = 0
    P[:,frag] = 0
    bath_idx = []
    env_idx = []
    e,U = np.linalg.eigh(P)
    nbath = 0.0
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            env_idx.append(nidx)
        elif thresh < ni < 1-thresh:
            if verbose > 1:
                print(" eigvalue: %12.8f" % ni)
            bath_idx.append(nidx)
            nbath += ni
        
            
    print(" # Electrons frag: %12.8f bath: %12.8f total: %12.8f" %(nfrag, nbath, nfrag+nbath))
    Cenv = Xinv@U[:,env_idx]
    Cbath = Xinv@U[:,bath_idx]
    
    
    C = np.hstack((Cfrag, Cbath))
    
    # Get virtual orbitals (these are just the orthogonal complement of the env and frag/bath
    if verbose > 1:
        print(" Now find span of virtual space")
    Q = np.eye(C.shape[0]) - X@C@C.T@X - X@Cenv@Cenv.T@X
    e,U = np.linalg.eigh(Q)
    vir_idx = []
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            vir_idx.append(nidx)
    Cvir = Xinv@U[:,vir_idx]

    assert(Cenv.shape[1] + Cvir.shape[1] + C.shape[1] == Nbas)

          
    # print(C.T@S@C)
    return (Cenv, C, Cvir)

def gram_schmidt(frags, S, thresh=1e-8):
    # |v'> = (1-sum_i |i><i|) |v>
    #      = |v> - sum_i |i><i|v>
    Nbas = S.shape[1]
    seen = []
    out = []
    seen = np.zeros((Nbas, 0))

    for f in frags:
        outf = np.zeros((Nbas, 0))
        # grab each orbital
        for fi in range(f.shape[1]):
            v = f[:,fi]
            v.shape = (Nbas, 1)

            # Compare to previous orbitals
            for fj in range(seen.shape[1]):
                j = seen[:,fj]
                j.shape = (Nbas, 1)
                ovlp = (j.T @ S @ v)[0]
                v = v - j * ovlp

                norm = np.sqrt((v.T @ S @ v)[0])
                if norm < thresh:
                    print(" Warning: small norm in GS: ", norm)
                v = v/norm
            
            outf = np.hstack((outf, v))
            seen = np.hstack((seen, v))
        out.append(outf)
    return out


In [21]:
# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d", "4s"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3d", "4s"):
            frag2.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2]
P = mf.make_rdm1()
Pa = P[0,:,:]
Pb = P[1,:,:]

C = mf.mo_coeff
S = mf.get_ovlp()

(Cenv, Cact, Cvir) = get_frag_bath(Pa, full, S)
pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Cact)

CC = np.hstack((Cenv, Cact, Cvir))

# Since we used Pa for the orbital partitioning, we will have an integer number of alpha electrons,
# but not beta. however, we can determine number of alpha because the env is closed shell.
n_tot_b = mf.nelec[1]
na_act = np.trace(Cact.T @ S @ Pa @ S @ Cact)
na_env = np.trace(Cenv.T @ S @ Pa @ S @ Cenv)
na_vir = np.trace(Cvir.T @ S @ Pa @ S @ Cvir)
nb_env = na_env
nb_act = n_tot_b - nb_env
nb_vir = na_vir
print(" # electrons: %12s %12s" %("α", "β"))
print("         Env: %12.8f %12.8f" %(na_env, nb_env))
print("         Act: %12.8f %12.8f" %(na_act, nb_act))
print("         Vir: %12.8f %12.8f" %(na_vir, nb_vir))

pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Cact)


Pa = Cact @ Cact.T @ S @ Pa @ S @ Cact @ Cact.T

(Ce1, Cf1, Cv1) = get_frag_bath(Pa, frag1, S)
(Ce2, Cf2, Cv2) = get_frag_bath(Pa, frag2, S)


frag_orbs = gram_schmidt((Cf1, Cf2), S)

Nbas = S.shape[1]
Ctot = np.zeros((Nbas,0))

clusters = []
init_fspace = []

ci_shift = 0
for fi,f in enumerate(frag_orbs):
    Ctot = np.hstack((Ctot, f))
    Paf = f.T @ S @ Pa @ S @ f
    Pbf = f.T @ S @ Pb @ S @ f
    
    na = np.trace(Paf)
    nb = np.trace(Pbf)
    
    clusters.append(list(range(ci_shift, ci_shift+f.shape[1])))
    ci_shift += f.shape[1]
    init_fspace.append((int(np.round(na)), int(np.round(nb))))
    print(" Fragment: %4i %s" %(fi,frags[fi]))
    print("    # α electrons: %12.8f" % na)
    print("    # β electrons: %12.8f" % nb)

    # pyscf.tools.molden.from_mo(mf.mol, "C_frag%2i.molden"%fi, f);

pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Ctot);


 Frag:  [3, 14, 15, 16, 17, 18, 34, 45, 46, 47, 48, 49]
 # Electrons frag:   8.83134616 bath:   3.16865384 total:  12.00000000

WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 10 11] not symmetrized, norm = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]

 # electrons:            α            β
         Env:  75.00000000  75.00000000
         Act:  12.00000000   2.00000000
         Vir:   0.00000000   0.00000000

WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 10 11] not symmetrized, norm = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]

 Frag:  [3, 14, 15, 16, 17, 18]
 # Electrons frag:   4.41567308 bath:   1.58432692 total:   6.00000000
 Frag:  [34, 45, 46, 47, 48, 49]
 # Electrons frag:   4.41567308 bath:   1.58432692 total:   6.00000000
 Fragment:    0 [3, 14, 15, 16, 17, 18]
    # α electrons:   6.00000000
    # β electrons:   1.18401614
 Fragment:    1 [34, 45, 46, 47, 48, 49]
    # α electrons:   6.00000000
    # β electrons:   1.18703054

WARN: orbitals [ 0  1  2  3  4  5  6  

In [22]:
print(clusters)
print(init_fspace)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
[(6, 1), (6, 1)]


# Make Integrals

In [23]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [24]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Ctot.T @ h @ Ctot;
j = Ctot.T @ j @ Ctot;
k = Ctot.T @ k @ Ctot;

In [25]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Ctot, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [26]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [27]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)